In [ ]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [ ]:
# mount your drive once per colab session
from google.colab import drive
drive.mount('/content/drive')
# !unzip '/content/drive/My Drive/train'.zip -d '/content/drive/My Drive/' # only need to do once for train and test

Mounted at /content/drive


In [ ]:
# unzip the test files!
#!unzip '/content/drive/My Drive/test'.zip -d '/content/drive/My Drive/' # only need this once

In [ ]:
# factor needed for cifar10
factor = int(256/32)
model = Sequential()
model.add(InputLayer(input_shape=(int(256 / factor), int(256 / factor), 1)))
model.add(Conv2D(64/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64/factor, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128/factor, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256/factor, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128/factor, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64/factor, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32/factor, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
# Get images, colab
X = []
i = 0
for filename in os.listdir('/content/drive/My Drive/train'):
  X.append(img_to_array(load_img('/content/drive/My Drive/train/'+filename)))
    
X = np.array(X, dtype=float)

# Set up train and test data
Xtrain = X[:len(X) - 1]
Xtrain = 1.0/255*Xtrain


OSError: ignored

In [ ]:
# Set up train and test data
split = int(0.95*len(X))
Xtrain = X[:split]

#splitting validation
Xtrain = 1.0/255*Xtrain
split_idx = int(0.9*len(Xtrain))
Xvalid=Xtrain[split_idx:]
Xtrain=Xtrain[:split_idx]

# Image transformer
batch_size = 100
def train_generator(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

def validationGenerator(batch_size):
    for batch in datagen.flow(Xvalid, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# new stuff from Naveen
steps_per_epoch = int(len(Xtrain)/batch_size)
validation_steps = int(len(Xvalid)/batch_size)

In [ ]:
# Train model   

filepath = "/content/drive/My Drive/Model/model.h5"

#model.load_weights(filepath)

checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
callbacks_list = [es,checkpoint]

tensorboard = TensorBoard(log_dir="output/first_run")
model.fit(train_generator(batch_size), callbacks=callbacks_list, epochs=1000, steps_per_epoch=steps_per_epoch,
          validation_data=validationGenerator(batch_size), validation_steps=validation_steps)

In [ ]:
# Save model
model_json = model.to_json()
with open("/content/drive/My Drive/Model/model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("/content/drive/My Drive/Model/model.h5")

In [ ]:
# Test images for cifar-10
files_sorted = os.listdir('/content/drive/My Drive/test')
files_sorted = [x for x in files_sorted if x.endswith(".png")]
files_sorted.sort(key=lambda x:int(x.split(".")[0]))

color_me = []
#should be after your training data
for i in range(1,1000):
    filename = files_sorted[i]
    fname = train_folder_cifar + filename
    color_me.append(img_to_array(load_img(fname)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0 / 255 * color_me)[:, :, :, 0]
color_me = color_me.reshape(color_me.shape + (1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((int(256 / factor), int(256 / factor), 3))
    cur[:, :, 0] = color_me[i][:, :, 0]
    cur[:, :, 1:] = output[i]
    imsave("/content/drive/My Drive/Results/" + str(i+1) + ".png", lab2rgb(cur))